# RNN 기반 텍스트 생성 (언어모델)

In [1]:
corpus = ['경마장에 있는 말이 뛰고 있다',
        '그의 말이 법이다',
        '가는 말이 고와야 오는 말이 곱다']

## 1. 데이터 전처리

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

# tokenizing해서 indexing -> 단어 수 확인
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index)
max_feature = vocab_size +1
print(f'단어 집합의 크기 : {vocab_size}')
print(f'max_feature : {max_feature}')

단어 집합의 크기 : 11
max_feature : 12


In [7]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [10]:
# 언어 모델용 학습 데이터 만들기
sequences = list()

for sent in corpus:
    #integer encoding
    indexed_sent = tokenizer.texts_to_sequences([sent])[0]
    print(f'전체 문장 : {indexed_sent}')

    # 두번째 단어부터 시작해서 한 단어씩 추가해서 학습 데이터 생성
    for i in range(1, len(indexed_sent)):
        sequence = indexed_sent[:i+1]
        print(sequence)
        sequences.append(sequence)

print(f'학습 데이터 수 : {len(sequences)}')

전체 문장 : [2, 3, 1, 4, 5]
[2, 3]
[2, 3, 1]
[2, 3, 1, 4]
[2, 3, 1, 4, 5]
전체 문장 : [6, 1, 7]
[6, 1]
[6, 1, 7]
전체 문장 : [8, 1, 9, 10, 1, 11]
[8, 1]
[8, 1, 9]
[8, 1, 9, 10]
[8, 1, 9, 10, 1]
[8, 1, 9, 10, 1, 11]
학습 데이터 수 : 11


In [11]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 구하기
maxlen = 0
for sequence in sequences:
    if len(sequence) > maxlen:
        maxlen = len(sequence)

print(maxlen)

6


In [14]:
# 학습 데이터 padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen)
print(padded_sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [15]:
# 입력 데이터와 정답 데이터 분리
import numpy as np

np_sequences = np.array(padded_sequences)
X = np_sequences[:, :-1]
y = np_sequences[:,-1]
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [16]:
# 정답 데이터 단어 11개를 카테고리로 one-hot-encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2. 모델 학습

In [20]:
# 모델 구축
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

input_units = max_feature
embedding_dim = 10
rnn_units = 32
output_units = max_feature

model = Sequential()
model.add(Embedding(input_units, embedding_dim))
model.add(SimpleRNN(rnn_units))
model.add(Dense(output_units, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 10)          120       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                1376      
                                                                 
 dense_1 (Dense)             (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [22]:
rnn_lang_model = [
    Embedding(input_units, embedding_dim),
    #SimpleRNN(rnn_units),
    LSTM(rnn_units),
    Dense(output_units, activation='softmax')
]
model = Sequential(rnn_lang_model)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 10)          120       
                                                                 
 lstm_1 (LSTM)               (None, 32)                5504      
                                                                 
 dense_3 (Dense)             (None, 12)                396       
                                                                 
Total params: 6,020
Trainable params: 6,020
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
# 모델 학습
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 0s 8ms/step - loss: 0.5651 - accuracy: 0.9091
Epoch 2/200
1/1 [==============================] - 0s 10ms/step - loss: 0.5582 - accuracy: 0.9091
Epoch 3/200
1/1 [==============================] - 0s 9ms/step - loss: 0.5514 - accuracy: 0.9091
Epoch 4/200
1/1 [==============================] - 0s 15ms/step - loss: 0.5447 - accuracy: 0.9091
Epoch 5/200
1/1 [==============================] - 0s 27ms/step - loss: 0.5380 - accuracy: 0.9091
Epoch 6/200
1/1 [==============================] - 0s 12ms/step - loss: 0.5315 - accuracy: 0.9091
Epoch 7/200
1/1 [==============================] - 0s 11ms/step - loss: 0.5251 - accuracy: 0.9091
Epoch 8/200
1/1 [==============================] - 0s 8ms/step - loss: 0.5188 - accuracy: 0.9091
Epoch 9/200
1/1 [==============================] - 0s 20ms/step - loss: 0.5125 - accuracy: 0.9091
Epoch 10/200
1/1 [==============================] - 0s 7ms/step - loss: 0.5065 - accuracy: 0.9091
Epoch 11/200
1/1 [=====

## 3. 첫 단어 입력 후 문장 생성

In [34]:
#단어를 입력하고, 반복할 회수를 주면 문장 생성
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, tokenizer, init_word, n): 
    current_word = init_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # Integer Encoding
        sequence = tokenizer.texts_to_sequences([current_word])
        # Padding
        padded_sequence = pad_sequences(sequence, maxlen=maxlen-1)
        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(padded_sequence, verbose=0)
        result_index = np.argmax(result)
        # 예측한 인덱스의 단어 가져오기
        word = tokenizer.index_word[result_index]
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word +' '+ word
        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word
    return init_word + sentence
sentence_generation(model, tokenizer, '경마장에', 4)

'경마장에 말이 말이 뛰고 있다'

In [47]:
sentence_generation(model, tokenizer, input('시작 단어열 입력 : '), int(input('생성 단어 수 : ')))

시작 단어열 입력 :  경마장에
생성 단어 수 :  4


'경마장에 있는 말이 뛰고 있다'